In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from sklearn.cluster import KMeans
from skimage import io, img_as_float

1) Загрузите картинку parrots.jpg. Преобразуйте изображение, приведя все значения в интервал от 0 до 1. Для этого можно воспользоваться функцией img_as_float из модуля skimage. Обратите внимание на этот шаг, так как при работе с исходным изображением вы получите некорректный результат.

In [2]:
parrots = io.imread('parrots.jpg')
parrots_values = img_as_float(parrots)
w, h, d = parrots_values.shape

2) Создайте матрицу объекты-признаки: характеризуйте каждый пиксель тремя координатами - значениями интенсивности в пространстве RGB.

In [3]:
pixels = pd.DataFrame(np.reshape(parrots_values, (w * h, d)), columns=['R', 'G', 'B'])
pixels.head(3)

,R,G,B
0,0.015686,0.494118,0.019608
1,0.007843,0.494118,0.007843
2,0.007843,0.494118,0.007843


3) Запустите алгоритм K-Means с параметрами init='k-means++' и random_state=241. После выделения кластеров все пиксели, отнесенные в один кластер, попробуйте заполнить двумя способами: медианным и средним цветом по кластеру.

In [4]:
def model_kmeans(pixels, n_clusters=8):
    
    pixels = pixels.copy()
    kmeans = KMeans(n_clusters=n_clusters, init='k-means++', random_state=241)
    pixels['cluster_number'] = kmeans.fit_predict(pixels)
    
    means = pixels.groupby('cluster_number').mean().values
    mean_pixels = [means[i] for i in pixels['cluster_number'].values]
    mean_image = np.reshape(mean_pixels, (w, h, d))
    io.imsave('mean_parrots.jpg', mean_image)
    
    medians = pixels.groupby('cluster_number').median().values
    medians_pixels = [medians[i] for i in pixels['cluster_number'].values]
    median_image = np.reshape(medians_pixels, (w, h, d))
    io.imsave('median_parrots.jpg', median_image)
    
    return mean_image, median_image

4) Измерьте качество получившейся сегментации с помощью метрики PSNR. Эту метрику нужно реализовать самостоятельно (см. определение).

In [5]:
def PSNR(image1, image2):
    mse = np.mean((image1 - image2) ** 2)
    return 10 * np.log10(1.0 / mse)

5) Найдите минимальное количество кластеров, при котором значение PSNR выше 20 (можно рассмотреть не более 20 кластеров, но не забудьте рассмотреть оба способа заполнения пикселей одного кластера). Это число и будет ответом в данной задаче.

In [7]:
for n in range(1, 21):
    mean_image, median_image = model_kmeans(pixels, n)
    psnr_mean, psnr_median = PSNR(parrots_values, mean_image), PSNR(parrots_values, median_image)
    print(psnr_mean, psnr_median)
    
    if psnr_mean > 20 or psnr_median > 20:
        print('N_clusters =', n)
        break

9.81883554477 9.42784046051
12.0801770849 11.6550772669
13.1534870446 12.7749481751
14.3678179409 14.0059287219
15.5271461973 15.1779090773
16.5439940966 16.0511767785
17.6409925086 17.3393684914
18.4368290141 18.1379714434
19.1126727358 18.8138873562
19.6407566359 19.4174725155
20.1275887465 19.8412840739
N_clusters = 11
